# Plots
For the plots in section 5: "Results and dicsuccsion".
## Physical properties of fluids and define functions
Define the used functions and the the physical properties of two working fluids: water and 50% Glycerin-water mixture.

In [ ]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.colors as mcolors
import itertools

data_directory = 'Data/'
figure_directory = 'Figure/'

# Physical properties of water at 20°C
rho_water = 998 # water density in kg/m3
mu_water = 0.0010005 # dynamic viscosity in Pa*s
sigma_air_water = 72.74e-3

# Physical properties of 50% Glycerin-water mixture at 20°C
rho_glycerin = 1142
mu_glycerin = 0.008026
sigma_air_glycerin = 68.12e-3

cos = np.cos
tanh = np.tanh
arccos = np.arccos
arcsinh = np.arcsinh

# Function to calculate the capillary number of contact line
def cal_caCL(uCL, mu, sigma):
    caCL = mu * uCL / sigma
    return caCL

# Function to calculate the dynamic contact angle with MKT model, T=20°C
def cal_theta_MKT(uCL, mu, sigma, theta_0, lamda, kappa0):
    cos_theta_MKT = cos(theta_0) - 2*1.380649e-23*295.15 / (sigma*lamda**2) * arcsinh(uCL/(2*kappa0*lamda))
    theta_MKT = arccos(cos_theta_MKT)
    theta_MKT = np.degrees(theta_MKT)
    return theta_MKT

def create_tick_labels(channels, fluids):
    """
    For violin plots, create tick labels for x-axis. Each label is composed
    of the employed mesh resolutions and a solver name.
    """
    fluid_label = ""
    for fluid in fluids:
        fluid_label = fluid_label + str(fluid) + ' '
    fluid_label = fluid_label.replace(' ', '$\quad\quad$') + '\n'
    tick_labels = [fluid_label + channel for channel in channels]
    return tick_labels

def assign_styles(unique_values):
    """
    Assign each value in 'unique_values' a marker and line style.
    Returns mappings as two dictionaries.
    """
    unique_values.sort()

    markers = itertools.cycle(['o', 'x', '+', 's', 'd', '*', '^', 'v'])
    colors = itertools.cycle(['o', 'x', '+', 's', 'd', '*', '^', 'v'])
    # Reserve solid line style for reference solutions
    lines = itertools.cycle(['dotted', 'dashed', 'dashdot', (0, (3, 1, 1, 1))])

    markerdict = dict()
    linestyledict = dict()

    for value in unique_values:
        markerdict[value] = next(markers)
        linestyledict[value] = next(lines)

    return markerdict, linestyledict

def creat_single_violing_plot(df, error_metrics, yscale='log',
                        ylabel='', file_name_addendum='', groupbyparameters=None):
    
    # Prepare color dictionaries for consistent coloring
    color_list_full = ['steelblue', 'red', 'mediumseagreen', 'gold', 'hotpink', 'sandybrown'] + \
                 [color for color in mcolors.CSS4_COLORS]
    darker_color_list_full = ['darkblue', 'darkred', 'darkgreen', 'darkgoldenrod', 'mediumvioletred', 'sienna'] + \
                        [color for color in mcolors.CSS4_COLORS][1:]

    channel_list = ([channel for channel, _ in df.groupby(level=["channel-fluid"])]) 
    color_list = color_list_full[:len(channel_list)] 
    darker_color_list = darker_color_list_full[:len(channel_list)]
    color_dict = {k:[v1,v2] for k,v1,v2 in zip(channel_list,color_list,darker_color_list)}
    
    tick_vector = []
    fluids = []
    ax = plt.axes() 
    
    for channel, channeldf in df.groupby(level=["channel-fluid"]):
        # Number of different resolutions by creating a set as intermediate step
        #nchannel = len(set(channeldf.index.get_level_values('fluid')))
        nchannel = 1
        tick_vector.append(channel)
        xpos = [x + nchannel*(len(tick_vector)-1) for x in range(1,nchannel+1)]
        #xpos = [len(tick_vector)]
        error_metrics.sort() # make sure the order is always the same
        for metric_count, error_metric in enumerate(error_metrics):
            plotcolumns = []
            plotcolumns.append(channeldf[error_metric])
            violin_parts = ax.violinplot(plotcolumns, positions=xpos, showmeans=True,
                                             widths=0.8, showmedians=False, showextrema=True)
            current_color = color_dict[channel][metric_count % 2] # modulus repeats colors for more than two metrics
            
            for partname in ('cbars','cmins','cmaxes','cmeans'):
                vp = violin_parts[partname]
                vp.set_edgecolor(current_color)
                vp.set_linewidth(1)
            '''
            vp = violin_parts['cmeans']
            vp.set_edgecolor(current_color)
            vp.set_linewidth(1)
            '''
            for pc in violin_parts['bodies']:
                pc.set_facecolor(current_color)
                pc.set_edgecolor(current_color)

        tick_positions = [nchannel*x+1 for x in range(len(tick_vector))]
        
        #print(tick_vector)
        tick_labels = create_tick_labels(tick_vector, fluids)
        ax.set_xticks(tick_positions)
        ax.set_xticklabels(tick_labels)
        ax.set_xticks(range(1, nchannel*len(tick_vector)+1), minor=True)
        #ax.grid(which="major")
        ax.set_ylabel(ylabel)

        if not os.path.isdir(figure_directory):
            os.mkdir(figure_directory)
        plt.savefig(os.path.join(figure_directory, "violin_" + file_name_addendum ), bbox_inches="tight")

## Violin plots for validation of automated image analysis
The violin plots for the validation of automated analysis in section 5.1:
- Interface detection (Fig. 9): `u_error` 
- Contact angle measurement (Fig. 10): `theta_error` 

In [ ]:
figure(figsize=(8, 3), dpi=300)

# Load the csv file
df_error = pd.read_csv(os.path.join(data_directory, "data_violin_plots.csv"), sep=';')

# Metadata columns
index_column = ['channel-fluid']
df_error.set_index(index_column, inplace=True)
df_error.sort_index(inplace=True)

# Data columns: "u_error" for velocity error, "theta_error" for dynamic contact angle error
data_column = ['theta_error']

# Error metrics for which plots are created
error_metrics = list(data_column)
groupbyviolin = index_column

y_label_u = r'Interface velocity error $|U_{theo}-U_{code}|/U_{theo}$'
y_label_theta = r'Contact angle error $|\theta_{manual}-\theta_{code}|/\theta_{manual}$'

creat_single_violing_plot(df_error, error_metrics, yscale='linear',
                    ylabel=y_label_theta, file_name_addendum='theta_error',
                    groupbyparameters=groupbyviolin)

## Straight channel with 50% Glycein-water mixture and three curved channels with water
For Fig. 11.

| Microchannel - Fluid | $\theta_0$ (deg) | $\lambda$ (nm) | $\kappa^0$ (kHz) |
| :--------: | :--------: | :--------: | :--------: |
| Straight - 50% Glycerin-water mixture | 82 | 0.95 | 1162.81 |
| Variation 1 - water | 87 | 0.81 | 756.56 |
| Variation 2 - water | 84 | 1.55 | 11.94 |
| Variation 3 - water | 86 | 1.07 | 284.86 |

In [ ]:
figure(figsize=(9, 4), dpi=300)

label = ['straight-glycerin', 'var1-water', 'var2-water', 'var3-water']
# Measured static contact angles of water in straight channel and three curved channels: var1, var2 and var3
theta0 = [82, 87, 84, 86]
# Fitting parameters lambda and kappa_0 of water in straight channel and three curved channels: var1, var2 and var3
lambdaP = [0.95e-9, 0.81e-9, 1.55e-9, 1.07e-9]
kappa0 = [1162.81e3, 756.56e3, 11.94e3, 284.86e3]

# Load the csv for straight channel and three curved channels with water
df = pd.read_csv("Data/data_full_dataset.csv", sep=';')
index_column = ['channel-fluid']
df.set_index(index_column, inplace=True)
df.sort_index(inplace=True)

group_by_parameters = list(index_column)
unique_values = list(df.index.unique())
markers, linestyles = assign_styles(unique_values)

for params, subsetdf in  df.groupby(level=group_by_parameters):
    
    plt.scatter(subsetdf['ca'], subsetdf['theta_code'], marker=markers[params], label=params)
    plt.xscale('log')
    plt.xlabel('Capillary number $Ca$')
    plt.ylabel(r'Dynamic contact angle $\theta_d$ [degree]')
    plt.legend()
    
uCL = np.linspace(0.00005, 0.023, 1000)
plt.plot(cal_caCL(uCL, mu_glycerin, sigma_air_glycerin), 
             cal_theta_MKT(uCL, mu_glycerin, sigma_air_glycerin, theta0[0]/180*np.pi, lambdaP[0], kappa0[0]), 
             label='MKT - '+label[0])

for i in range(0,3,1):
    plt.plot(cal_caCL(uCL, mu_water, sigma_air_water), 
             cal_theta_MKT(uCL, mu_water, sigma_air_water, theta0[i+1]/180*np.pi, lambdaP[i+1], kappa0[i+1]), 
             label='MKT - '+ label[i+1])
    plt.legend()
    
plt.savefig(os.path.join(figure_directory, "mkt_full_dataset"), bbox_inches="tight")

## Different parts of variation 3: straight, position 1 and position 2
For Fig. 13.

| Channel part | $\theta_0$ (deg) | $\lambda$ (nm) | $\kappa^0$ (kHz) |
| :--------: | :--------: | :--------: | :--------: |
| Inside position 1 | 70 | 1.03 | 424.12 |
| Inside position 2 | 86 | 1.20 | 195.86 |
| Outside position 1 | 86 | 1.03 | 400.75 |
| Outside position 2 | 86 | 1.02 | 410.16 |
| Straight | 86 | 1.07 | 284.86 |

In [ ]:
figure(figsize=(8, 3), dpi=300)

label_var3 = ['inside1', 'inside2', 'outside1', 'outside2', 'straight']
# Measured static contact angles of water in different parts of variation 3: inside position 1, inside position 2, outside position1, outside position 2 and straight part
theta0_var3 = [70, 86, 86, 86, 86]
# Fitting parameters lambda and kappa_0 of water  in different parts of variation 3: inside position 1, inside position 2, outside position1, outside position 2 and straight part
lambda_var3 = [1.03e-9, 1.20e-9, 1.03e-9,  1.02e-9, 1.07e-9]
kappa0_var3 = [424.12e3, 400.75e3, 400.75e3, 410.16e3, 284.86e3]

# Load the csv for variation 3
df_var3 = pd.read_csv("Data/data_variation3_part.csv", sep=';')
index_column_var3 = ['part']
df_var3.set_index(index_column_var3, inplace=True)
df_var3.sort_index(inplace=True)

group_by_parameters = list(index_column_var3)
unique_values = list(df_var3.index.unique())
markers, linestyles = assign_styles(unique_values)

for params, subsetdf in  df_var3.groupby(level=group_by_parameters):
    
    plt.scatter(subsetdf['ca'], subsetdf['theta_code'], marker=markers[params], label=params)
    plt.xscale('log')
    plt.xlabel('Capillary number $Ca$')
    plt.ylabel(r'Dynamic contact angle $\theta_d$ [degree]')
    plt.legend()
    
for i in range(len(theta0_var3)):
    
    uCL = np.linspace(0.00001, 0.023, 1000)
    plt.plot(cal_caCL(uCL, mu_water, sigma_air_water), 
             cal_theta_MKT(uCL, mu_water, sigma_air_water, theta0_var3[i]/180*np.pi, lambda_var3[i], kappa0_var3[i]), 
             label='MKT - '+label_var3[i])
    plt.legend()
    
plt.savefig(os.path.join(figure_directory, "variation3_part"), bbox_inches="tight")